### Film и crop в массив

на основе scikit-video

In [ ]:
!pip install scikit-video -q
!pip install einops -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 773.6 kB/s eta 0:00:00


# common_metrics_on_video_quality
https://github.com/JunyaoHu/common_metrics_on_video_quality/tree/main

In [ ]:
!pip install lpips -q
!git clone https://github.com/JunyaoHu/common_metrics_on_video_quality.git -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.1 MB/s eta 0:00:00


In [ ]:
import os
import torch
import sys
dir_project = '/content/common_metrics_on_video_quality'
sys.path.append(dir_project)

import skvideo.io
import numpy as np
np.float = np.float64
np.int = np.int_
import json
from calculate_fvd import calculate_fvd
from calculate_psnr import calculate_psnr
from calculate_ssim import calculate_ssim
from calculate_lpips import calculate_lpips
device = torch.device("cuda")

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [on]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 95.1MB/s]


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/alex.pth


In [ ]:
MAIN_DIR = " you main dir"
EVAL_DIR = MAIN_DIR + "/Increment_spliter_next"

MOVI_DIR = MAIN_DIR + "/MSR_VIT_movi"
ORIG_DIR = MOVI_DIR + "/crop_hand_selected_mp4"

SIZE = 256
QTYFRAMES = 16

def movi_numpy_to_tensor(array):
    return torch.tensor(array).permute(0, 3, 1, 2).unsqueeze(0)

## real_video

In [ ]:
video_list = os.listdir(ORIG_DIR)
video_list.sort()
print(len(video_list))

video_directory = ORIG_DIR +"/"

all_video = []
for film in video_list:
    vid = skvideo.io.vread(fname = video_directory + film,
                           num_frames = QTYFRAMES,
                           outputdict={
                              "-sws_flags": "bilinear",
                              "-s": f"{SIZE}x{SIZE}"
                          }
    )
    all_video.append(movi_numpy_to_tensor(vid))

real_video = torch.concat(all_video)
real_video.shape

31


torch.Size([31, 16, 3, 256, 256])

#  Fréchet Video Distance (FVD)

## predict_step_vectors_H512_W512_F16_S75_caption_uniform

In [ ]:
GEN_DIR = EVAL_DIR + "/predict_step_vectors_H512_W512_F16_S75_caption_uniform"
video_list = os.listdir(GEN_DIR)
video_list.sort()
print(len(video_list))

video_directory = GEN_DIR +"/"

all_video = []
for film in video_list:
    vid = skvideo.io.vread(fname = video_directory + film,
                           num_frames = QTYFRAMES,
                           outputdict={
                              "-sws_flags": "bilinear",
                              "-s": f"{SIZE}x{SIZE}"
                          }
    )
    all_video.append(movi_numpy_to_tensor(vid))

predict_step_video = torch.concat(all_video)
predict_step_video.shape

31


torch.Size([31, 16, 3, 256, 256])

In [ ]:
result = {}
result['fvd_VGPT'] = calculate_fvd(real_video/255, predict_step_video/255, device, method='videogpt')
result['fvd_GAN'] = calculate_fvd(real_video/255, predict_step_video/255, device, method='styleganv')
result['ssim'] = calculate_ssim(real_video/255, predict_step_video/255)
result['psnr'] = calculate_psnr(real_video/255, predict_step_video/255)
result['lpips'] = calculate_lpips(real_video/255, predict_step_video/255, device)
print(json.dumps(result, indent=4))

calculate_fvd...
/content/common_metrics_on_video_quality/fvd/videogpt/i3d_pretrained_400.pt


100%|██████████| 7/7 [00:33<00:00,  4.74s/it]


calculate_fvd...
/content/common_metrics_on_video_quality/fvd/styleganv/i3d_torchscript.pt


100%|██████████| 7/7 [00:26<00:00,  3.81s/it]


calculate_ssim...


...

calculate_psnr...


100%|██████████| 31/31 [00:00<00:00, 146.04it/s]


calculate_lpips...


100%|██████████| 31/31 [00:02<00:00, 14.85it/s]


{
    "fvd_VGPT": {
        "value": {
            "10": 3280.9501953125,
            "11": 2649.98974609375,
            "12": 3348.17578125,
            "13": 2219.3583984375,
            "14": 2648.40576171875,
            "15": 2221.918212890625,
            "16": 2510.48681640625
        },
        "video_setting": [
            31,
            3,
            16,
            256,
            256
        ],
        "video_setting_name": "batch_size, channel, time, heigth, width"
    },
    "fvd_GAN": {
        "value": {
            "10": 3377699720528434.5,
            "11": -351843720887894.8,
            "12": -14155220.329529464,
            "13": 805306727.5649868,
            "14": -29686813949565.734,
            "15": -105226698427.88438,
            "16": -10995116277389.26
        },
        "video_setting": [
            31,
            3,
            16,
            256,
            256
        ],
        "video_setting_name": "batch_size, channel, time, heigth, width"


In [ ]:
import numpy as np

In [ ]:
d = {       "10": 3280.9501953125,
            "11": 2649.98974609375,
            "12": 3348.17578125,
            "13": 2219.3583984375,
            "14": 2648.40576171875,
            "15": 2221.918212890625,
            "16": 2510.48681640625
                 }

res = np.array(list(d.values()))
print(f"FVD predict_step_video mean: {np.mean(res )}")
print(f"FVD predict_step_video std: {np.std(res )}")

FVD predict_step_video mean: 2697.040701729911
FVD predict_step_video std: 423.96604525473964


In [ ]:
d = {
           "0": 0.23056031884738087,
            "1": 0.23123363367541577,
            "2": 0.22588996160876065,
            "3": 0.22298685651889188,
            "4": 0.2317013687326669,
            "5": 0.22449647202884748,
            "6": 0.22164116354176164,
            "7": 0.2205403414677309,
            "8": 0.22370477510452977,
            "9": 0.22561460241295753,
            "10": 0.22263446883449747,
            "11": 0.2251050069086525,
            "12": 0.2221486970350158,
            "13": 0.22455431374398319,
            "14": 0.220901946673664,
            "15": 0.21961134229974497
                 }

res = np.array(list(d.values()))
print(f"SSIM predict_step_video mean: {np.mean(res )}")
print(f"SSIM  predict_step_video std: {np.std(res )}")

SSIM predict_step_video mean: 0.22458282933965634
SSIM  predict_step_video std: 0.0036227594495421103


In [ ]:
d = {
            "0": 8.406304762329766,
            "1": 8.39883904917015,
            "2": 8.163262878435756,
            "3": 7.9376453860877865,
            "4": 7.9786802076915775,
            "5": 8.11341174682033,
            "6": 8.126364942966342,
            "7": 8.14343516683433,
            "8": 8.24124348651807,
            "9": 8.322475274498574,
            "10": 8.145031509436594,
            "11": 8.137026407547985,
            "12": 8.150443828924542,
            "13": 8.141127118115909,
            "14": 8.037128126183493,
            "15": 7.914551372420162
                 }

res = np.array(list(d.values()))
print(f"PSNR predict_step_video mean: {np.mean(res )}")
print(f"PSNR  predict_step_video std: {np.std(res )}")

PSNR predict_step_video mean: 8.147310703998835
PSNR  predict_step_video std: 0.1393241930544055


In [ ]:
d = {
            "0": 0.7775549965520059,
            "1": 0.7820272734088283,
            "2": 0.7852610541928199,
            "3": 0.7953670582463664,
            "4": 0.8062487898334381,
            "5": 0.8137423838338544,
            "6": 0.8181287338656764,
            "7": 0.8184390721782562,
            "8": 0.8203787419103807,
            "9": 0.816673796023092,
            "10": 0.8151470249699008,
            "11": 0.8174519750379747,
            "12": 0.8167855605002372,
            "13": 0.8197166073706842,
            "14": 0.818873405456543,
            "15": 0.8178102893214072
                 }

res = np.array(list(d.values()))
print(f"LPIPS predict_step_video mean: {np.mean(res )}")
print(f"LPIPS  predict_step_video std: {np.std(res )}")

LPIPS predict_step_video mean: 0.8087254226688416
LPIPS  predict_step_video std: 0.014394322848955985


# Frechet Inception Distance (FID)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install -q torchmetrics[image]
from torchmetrics.image.fid import FrechetInceptionDistance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 71.2 MB/s eta 0:00:00


## predict_step_vectors_H512_W512_F16_S75_caption_uniform

In [ ]:
fid_eval = []
for i in range(real_video.shape[0]):
    fid = FrechetInceptionDistance(normalize=True)
    real_images = real_video[i]
    fake_images = predict_step_video[i]
    fid.update(real_images, real=True)
    fid.update(fake_images, real=False)
    eval = float(fid.compute())
    fid_eval.append(eval)
    print(f"FID: {eval}")

fid_eval = np.array(fid_eval)
print(f"FID predict_step_video mean: {np.mean(fid_eval)}")
print(f"FID predict_step_video std: {np.std(fid_eval)}")

Downloading: "https://github.com/toshas/torch-fidelity/releases/download/v0.2.0/weights-inception-2015-12-05-6726825d.pth" to /root/.cache/torch/hub/checkpoints/weights-inception-2015-12-05-6726825d.pth
100%|██████████| 91.2M/91.2M [00:00<00:00, 437MB/s]


FID: 323.722412109375
FID: 481.73724365234375
FID: 540.8685913085938
FID: 467.3667297363281
FID: 415.52178955078125
FID: 308.6086730957031
FID: 285.4802551269531
FID: 491.8378601074219
FID: 402.7022705078125
FID: 495.6676330566406
FID: 376.2535400390625
FID: 415.42340087890625
FID: 354.29034423828125
FID: 617.343017578125
FID: 484.9372863769531
FID: 364.16644287109375
FID: 417.5959167480469
FID: 390.7661437988281
FID: 364.2007141113281
FID: 382.1861572265625
FID: 432.3722229003906
FID: 377.99969482421875
FID: 449.6792907714844
FID: 495.21795654296875
FID: 340.0849304199219
FID: 680.5221557617188
FID: 400.8719482421875
FID: 473.537109375
FID: 514.5734252929688
FID: 407.52410888671875
FID: 394.6655578613281
FID predict_step_video mean: <built-in method mean of numpy.ndarray object at 0x7bd36c247c90>
FID predict_step_video std: 84.5863966318969


In [ ]:
print(f"FID predict_step_video mean: {np.mean(fid_eval)}")
print(f"FID predict_step_video std: {np.std(fid_eval)}")

FID predict_step_video mean: 430.57176848380794
FID predict_step_video std: 84.5863966318969
